In [50]:
import os
from pprint import pprint

import numpy as np
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import nltk

import json
load_dotenv()
import cohere

In [51]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/konak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [52]:
# model = SentenceTruansformer('all-MiniLM-L6-v2', device="cpu")
co = cohere.Client(os.getenv("COHERE_API_KEY"))
response = co.embed(["hello", "goodbye"])
# print('Embeddings: {}'.format(response.embeddings))

In [53]:
data = json.load(open('data.json'))

In [55]:
vectors = []
batch_size = 64
batch = []
print(len(data))
for row in data:
    batch.append(row['text'])
    if len(batch) >= batch_size:
        vectors.append(co.embed(batch).embeddings)  # Text -> vector encoding happens here
        batch = []

if len(batch) > 0:
    vectors.append(co.embed(batch).embeddings)
    batch = []
print(len(vectors), len(vectors[0]), len(vectors[1]))

vectors = np.concatenate(vectors)

124
2 64 60


In [56]:
vectors.shape

(124, 4096)

In [57]:
np.save('vectors.npy', vectors, allow_pickle=False)

# Test Query
The answer is
  - $ 3 + 5 = 8$
  - $ 3 + 4 = 7$


In [58]:
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
# Take a random description as a query
sample_query = "lower costs"
print(sample_query)

lower costs


In [63]:
query_vector = co.embed([sample_query]).embeddings[0] # Convert query description into a vector.

In [64]:
scores = cosine_similarity([query_vector], vectors)[0]  # Look for the most similar vectors, manually score all vectors
top_scores_ids = np.argsort(scores)[-5:][::-1]  # Select top-5 with vectors the largest scores

In [65]:
# Check if result similar to the query
for top_id in top_scores_ids:
    print(data[top_id])
    # print(df.iloc[top_id].description)
    print("-----")


{'text': 'By committing to use instances for longer periods, you can reduce costs and get more predictable pricing.', 'meta': {'source': 'AWSGuide.md'}}
-----
{'text': 'The operating expenses for the quarter were $6 million, which are more than last year.', 'meta': {'source': 'quarterly.md'}}
-----
{'text': 'You can then make adjustments, such as reducing the number of virtual machines running or resizing instances, to optimize resource usage and minimize costs.', 'meta': {'source': 'AWSGuide.md'}}
-----
{'text': 'Also, make sure to review and optimize data transfer costs, especially for large file transfers or for data transfer between different regions or providers.', 'meta': {'source': 'AWSGuide.md'}}
-----
{'text': '## Response:\nIn an effort to cut costs we’ve made the decision to reduce headcount.', 'meta': {'source': 'quarterly.md'}}
-----
